# Video Detection Demo with PytorchWildlife

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/16-OjFVQ6nopuP-gfqofYBBY00oIgbcr1?usp=sharing)


This tutorial guides you on how to use PyTorchWildlife for video detection and classification. We will go through the process of setting up the environment, defining the detection and classification models, as well as performing inference and saving the results in an annotated video.

## Prerequisites
Install PytorchWildlife running the following commands

Also, make sure you have a CUDA-capable GPU if you intend to run the model on a GPU. In Google Colab, click on Runtime in the menu bar, select Change runtime type, choose GPU under Hardware accelerator, and then click Save.

This notebook can also run on CPU.


In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.8 python3.8-dev python3.8-distutils libpython3.8-dev

#change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

#Check that it points at the right location
!python3 --version

In [ ]:
# install pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall

#install colab's dependencies
!python3 -m pip install ipython ipython_genutils ipykernel jupyter_console prompt_toolkit httplib2 astor

In [ ]:
# link to the old google package
!ln -s /usr/local/lib/python3.10/dist-packages/google \
       /usr/local/lib/python3.8/dist-packages/google

!sed -i "s/from IPython.utils import traitlets as _traitlets/import traitlets as _traitlets/" /usr/local/lib/python3.8/dist-packages/google/colab/*.py
!sed -i "s/from IPython.utils import traitlets/import traitlets/" /usr/local/lib/python3.8/dist-packages/google/colab/*.py

In [ ]:
#Install PytorchWildlife
!pip install pytorchwildlife

Now, you must go to Runtime and restart your session to continue.

## Importing libraries
First, let's import the necessary libraries and modules.

In [ ]:
import os
import numpy as np
import supervision as sv
import torch
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.models import classification as pw_classification
from PytorchWildlife import utils as pw_utils

## Model Initialization
We'll  define the device to run the models and then we will initialize the models for both video detection and classification.

In [ ]:
!wget -O repo.zip https://github.com/microsoft/CameraTraps/archive/refs/heads/main.zip
!unzip -q repo.zip -d ./

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda":
    torch.cuda.set_device(0)

SOURCE_VIDEO_PATH = "/content/CameraTraps-main/demo/demo_data/videos/opossum_example.MP4"
TARGET_VIDEO_PATH = "/content/CameraTraps-main/demo/demo_data/videos/opossum_processed.MP4"

# Verify the checkpoints directory exists to save the model weights if pretrained=True
os.makedirs(os.path.join(torch.hub.get_dir(), "checkpoints"), exist_ok=True)

# Initializing the MegaDetectorV6 model for image detection
detection_model = pw_detection.MegaDetectorV6(device=DEVICE, pretrained=True, version="yolov9c")

# Uncomment the following line to use MegaDetectorV5 instead of MegaDetectorV6
#detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True, version="a")

# Initializing the AI4GOpossum model for image classification
classification_model = pw_classification.AI4GOpossum(device=DEVICE, pretrained=True)

## Video Processing
For each frame in the video, we'll apply detection and classification, and then annotate the frame with the results. The processed video will be saved with annotated detections and classifications.

In [ ]:
box_annotator = sv.BoxAnnotator(thickness=4)
lab_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK, text_thickness=4, text_scale=2)

def callback(frame: np.ndarray, index: int) -> np.ndarray:
    results_det = detection_model.single_image_detection(frame, img_path=index)
    labels = []
    for xyxy in results_det["detections"].xyxy:
        cropped_image = sv.crop_image(image=frame, xyxy=xyxy)
        results_clf = classification_model.single_image_classification(cropped_image)
        labels.append("{} {:.2f}".format(results_clf["prediction"], results_clf["confidence"]))
    annotated_frame = lab_annotator.annotate(
        scene=box_annotator.annotate(
            scene=frame,
            detections=results_det["detections"],
        ),
        detections=results_det["detections"],
        labels=labels,
    )
    return annotated_frame 

pw_utils.process_video(source_path=SOURCE_VIDEO_PATH, target_path=TARGET_VIDEO_PATH, callback=callback, target_fps=5)


### Copyright (c) Microsoft Corporation. All rights reserved.
### Licensed under the MIT License.